In [1]:
pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install flask

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymysql

db = pymysql.connect(
    host='localhost',
    user='root',
    password='yummy',
)

print("Connected to MySQL!")

cursor = db.cursor()

cursor.execute("CREATE DATABASE fitness")
db.commit()
db.close()

Connected to MySQL!


ProgrammingError: (1007, "Can't create database 'fitness'; database exists")

#### Execute Query

In [3]:
import pymysql

db = pymysql.connect(
    host='localhost',
    user='root',
    password='yummy',
    database='fitness'
)
cursor = db.cursor()
q1 = """
    CREATE TABLE Users (
    user_id INT AUTO_INCREMENT PRIMARY KEY,
    email VARCHAR(255) UNIQUE NOT NULL,
    name VARCHAR(255),
    age INT,
    height FLOAT,
    weight FLOAT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
);"""

q2 = """
    CREATE TABLE Workouts (
    workout_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT NOT NULL,
    workout_type VARCHAR(50),
    duration INT,
    calories_burned FLOAT,
    notes TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE CASCADE
);"""

q3 = """
    CREATE TABLE Exercises (
    exercise_id INT AUTO_INCREMENT PRIMARY KEY,
    workout_id INT NOT NULL,
    exercise_name VARCHAR(255),
    sets INT,
    reps_per_set INT,
    duration INT,
    calories_burned_per_exercise FLOAT,
    FOREIGN KEY (workout_id) REFERENCES Workouts(workout_id) ON DELETE CASCADE
);"""

q4 = """
    CREATE TABLE Goals (
    goal_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT NOT NULL,
    goal_type VARCHAR(50),
    target_value FLOAT,
    current_progress FLOAT,
    deadline DATE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    status VARCHAR(20),
    FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE CASCADE
);"""

q5 = """
    CREATE TABLE Progress (
    progress_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT NOT NULL,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    weight FLOAT,
    workout_count INT DEFAULT 0,
    calories_burned FLOAT DEFAULT 0,
    goal_id INT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE CASCADE,
    FOREIGN KEY (goal_id) REFERENCES Goals(goal_id) ON DELETE CASCADE
);"""

q6 = """
    CREATE TABLE Diet (
    diet_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT NOT NULL,
    meal_type VARCHAR(50),
    calories FLOAT,
    protein FLOAT,
    carbs FLOAT,
    fats FLOAT,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES Users(user_id) ON DELETE CASCADE
);"""

cursor.execute(q1)
cursor.execute(q2)
cursor.execute(q3)
cursor.execute(q4)
cursor.execute(q5)
cursor.execute(q6)
print("Success!")

db.commit()
db.close()

Success!


#### Sample Data

In [6]:
import pymysql

db = pymysql.connect(
    host='localhost',
    user='root',
    password='yummy',
    database='fitness'
)
cursor = db.cursor()

# Insert sample data into Users
q1 = """
    INSERT INTO Users (email, name, age, height, weight) 
    VALUES 
    ('john.doe@example.com', 'John Doe', 30, 175.0, 75.0),
    ('jane.smith@example.com', 'Jane Smith', 25, 160.0, 60.0),
    ('mike.jones@example.com', 'Mike Jones', 35, 180.0, 85.0);
"""

# Insert sample data into Workouts
q2 = """
    INSERT INTO Workouts (user_id, workout_type, duration, calories_burned, notes) 
    VALUES 
    (1, 'Running', 30, 300.0, 'Morning jog'),
    (2, 'Cycling', 45, 350.0, 'Weekend ride'),
    (3, 'Swimming', 60, 500.0, 'Long pool session');
"""

# Insert sample data into Exercises
q3 = """
    INSERT INTO Exercises (workout_id, exercise_name, sets, reps_per_set, duration, calories_burned_per_exercise)
    VALUES
    (1, 'Push-ups', 3, 15, 5, 50.0),
    (1, 'Squats', 3, 20, 5, 40.0),
    (2, 'Leg Press', 3, 12, 10, 70.0);
"""

# Insert sample data into Goals
q4 = """
    INSERT INTO Goals (user_id, goal_type, target_value, current_progress, deadline, status)
    VALUES 
    (1, 'Weight Loss', 70.0, 75.0, '2024-06-01', 'Active'),
    (2, 'Endurance', 60.0, 50.0, '2024-12-01', 'Active');
"""

# Insert sample data into Progress
q5 = """
    INSERT INTO Progress (user_id, weight, workout_count, calories_burned, goal_id) 
    VALUES 
    (1, 75.0, 5, 1500.0, 1),
    (2, 60.0, 8, 2500.0, 2);
"""

# Insert sample data into Diet
q6 = """
    INSERT INTO Diet (user_id, meal_type, calories, protein, carbs, fats) 
    VALUES 
    (1, 'Breakfast', 500, 30, 50, 15),
    (2, 'Lunch', 600, 40, 60, 20);
"""

# Execute queries to insert the sample data
cursor.execute(q1)
cursor.execute(q2)
cursor.execute(q3)
cursor.execute(q4)
cursor.execute(q5)
cursor.execute(q6)

# Commit the changes to the database
db.commit()

print("Sample data inserted successfully.")

# Close the connection
db.close()


Sample data inserted successfully.


#### Triggers


In [1]:
import pymysql

# Connect to the database
db = pymysql.connect(
    host='localhost',
    user='root',
    password='yummy',
    database='fitness'
)
cursor = db.cursor()

try:
    trigger_sql = """
    CREATE TRIGGER UpdateProgress AFTER INSERT ON Workouts
    FOR EACH ROW
    BEGIN
        UPDATE Progress
        SET workout_count = workout_count + 1, calories_burned = calories_burned + NEW.calories_burned
        WHERE user_id = NEW.user_id;
    END
    """
    cursor.execute(trigger_sql)
    db.commit()
    print("Trigger 'UpdateProgress' has been created.")
except Exception as e:
    print(f"Error: {e}")
finally:
    db.close()


Error: (1359, 'Trigger already exists')


#### Creating Procedure

In [7]:
import pymysql

db = pymysql.connect(
    host='localhost',
    user='root',
    password='yummy',
    database='fitness'
)
cursor = db.cursor()
try:
    procedure_sql = """
    CREATE PROCEDURE GetUserWorkouts (IN userId INT)
    BEGIN
        SELECT workout_id, workout_type, duration, calories_burned, notes
        FROM Workouts
        WHERE user_id = userId;
    END 
    """
    cursor.execute(procedure_sql)
    db.commit()
    print ("procedure created")
except Exception as e:
    print(f"Error occurred while creating procedure: {e}")
finally:
    db.close()

procedure created
